In [ ]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-istaden", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

In [ ]:
%%send_to_spark -i username -t str -n username

In [ ]:
#Loading the data 

In [ ]:
df = spark.read.orc('/data/sbb/orc/istdaten')

In [ ]:
df=df.selectExpr('betriebstag as date',
                 'produkt_id as transport_type',
                 'haltestellen_name as stop_name',
                 'ankunftszeit as arrival_scheduled',
                 'an_prognose as arrival_actual',
                 'durchfahrt_tf as stop_skip',
                 'bpuic as stop_id',
                  'linien_text as line_name',
                  'verkehrsmittel_text as line_type',
                  'linien_id as line_id'
                )

In [ ]:
df.select("stop_id").distinct().count()

In [ ]:
stop_id_reachable.select("stop_id").distinct().count()

In [ ]:
import pyspark.sql.functions as F

In [ ]:
#convert day of the trips between
df = df.withColumn('date', F.to_timestamp('date', "dd.MM.yyyy"))
df = df.withColumn('arrival_scheduled', F.to_timestamp('arrival_scheduled', "dd.MM.yyyy HH:mm"))
df = df.withColumn('arrival_actual', F.to_timestamp('arrival_actual', 'dd.MM.yyyy HH:mm:ss'))    

#keep only rows for stops that are not skipped
df=df.filter(df.stop_skip==False)

#keep only rows for stops during the week
df=df.withColumn("day_of_week",F.dayofweek(df.date))
df=df.filter(df.day_of_week.between(2,6))

#hours between 8am and 8pm
min_day_hour,max_day_hour=8,20
df = df.filter(F.hour(F.col('arrival_scheduled')).cast('int').between(min_day_hour, max_day_hour))

#only keep stops within the 15km radius
reachable_stops_path="/user/%s/final/parquet/reachable_stops" %username
stop_id_reachable=spark.read.parquet(reachable_stops_path)
stop_id_reachable=stop_id_reachable.withColumn("stop_id",F.split(stop_id_reachable.stop_id,':')[0]).drop("stop_name")

df=df.join(stop_id_reachable,on="stop_id")

#add hour column
df=df.withColumn("hour",F.hour(F.col("arrival_scheduled")))


In [ ]:
df.select("stop_id").distinct().count()

In [ ]:
#filter the rows where transport_type is null
df=df.where((F.col("transport_type")=="Tram") | (F.col("transport_type")=="Zug")|(F.col("transport_type")=="Bus"))

In [ ]:
stop_id_reachable.show(10)

In [ ]:
#computing actual delays
@F.udf
def relu(x):
    return max(x,0)
df=df.withColumn("delay",relu((F.col("arrival_actual").cast("long")-F.col("arrival_scheduled").cast("long"))))
df = df.where(~(F.col('delay').isNull()))

In [ ]:
#statistics type of transport average delays
transport_avg_delays=df.groupby("transport_type").agg(F.mean('delay'))
transport_avg_delays.show()

In [ ]:
transport_avg_delays=transport_avg_delays.withColumnRenamed("avg(delay)","avg_delay")

In [ ]:
transport_avg_delays.show()

In [ ]:
import matplotlib
matplotlib.use('agg')
import matplotlib.pylab as plt

plt.rcParams['figure.figsize'] = (30,8)
plt.rcParams['font.size'] = 12
plt.style.use('fivethirtyeight')

In [ ]:
import matplotlib.pyplot as plt
# Plot transport average delays

t_avg_delays_plot=transport_avg_delays.toPandas()

plt.bar(x=t_avg_delays_plot.transport_type,height=t_avg_delays_plot.avg_delay)
#transport_avg_delays["avg_delay"].plot.bar
plt.xlabel('average delay distribution by train')
plt.ylabel('average delay (seconds)')
plt.xticks(range(len(t_avg_delays_plot)), t_avg_delays_plot.transport_type)

In [ ]:
%matplot plt

In [ ]:
hour_avg_delays=df.groupby("hour").agg(F.mean('delay'))

hour_avg_delays=hour_avg_delays.withColumnRenamed("avg(delay)","avg_delay")

In [ ]:
# Plot transport average delays
plt.cla()
h_avg_delays_plot=hour_avg_delays.toPandas().sort_values(by="hour").reset_index(drop=True)

plt.bar(x=h_avg_delays_plot.hour, height=h_avg_delays_plot.avg_delay, color=(0.2, 0.4, 0.6, 0.6))
 
plt.xlabel('average delay distribution by hour')
plt.ylabel('average delay (seconds)')
plt.xticks(h_avg_delays_plot.hour)
# Show the graph
%matplot plt

In [ ]:
h_avg_delays_plot

In [ ]:
train_types=df.where(F.col("transport_type")=="Zug").groupby("line_name").agg(F.mean('delay'))
#train_types.filter(train_types.line_name[0:2]=="RE").show()
train_types.show()

In [ ]:
df

In [ ]:
train_types_stops=df.where(F.col("transport_type")=="Zug").groupby("line_name", "hour", "stop_id").agg(F.mean('delay'))
#train_types.filter(train_types.line_name[0:2]=="RE").show()
train_types_stops.show()

In [ ]:
train_types_hour=df.where(F.col("transport_type")=="Zug").groupby("line_name","hour").agg(F.mean('delay'))
train_types_hour.show()

In [ ]:
train_types_hour.groupby('line_name').count().show()

In [ ]:
bus_types=df.where(F.col("transport_type")=="Bus").groupby("line_name").agg(F.mean('delay'))
bus_types.show()

In [ ]:
bus_types_stops=df.where(F.col("transport_type")=="Bus").groupby("line_name", "hour", "stop_id").agg(F.mean('delay'))
bus_types_stops.show()

In [ ]:
bus_types_hour=df.where(F.col("transport_type")=="Bus").groupby("line_name", "hour").agg(F.mean('delay'))
bus_types_hour.show()

In [ ]:
tram_types=df.where(F.col("transport_type")=="Tram").groupby("line_name").agg(F.mean('delay'))
tram_types.show()

In [ ]:
tram_types_stops=df.where(F.col("transport_type")=="Tram").groupby("line_name", "hour", "stop_id").agg(F.mean('delay'))
tram_types_stops.show()

In [ ]:
tram_types_hour=df.where(F.col("transport_type")=="Tram").groupby("line_name", "hour").agg(F.mean('delay'))
tram_types_hour.show()

In [ ]:
#Renaming
train_types = train_types.withColumnRenamed("avg(delay)","avg_delay")
train_types_hour = train_types_hour.withColumnRenamed("avg(delay)","avg_delay")
train_types_stops = train_types_stops.withColumnRenamed("avg(delay)","avg_delay")

bus_types= bus_types.withColumnRenamed("avg(delay)","avg_delay")
bus_types_hour= bus_types_hour.withColumnRenamed("avg(delay)","avg_delay")
bus_types_stops = bus_types_stops.withColumnRenamed("avg(delay)","avg_delay")

tram_types = tram_types.withColumnRenamed("avg(delay)","avg_delay")
tram_types_hour = tram_types_hour.withColumnRenamed("avg(delay)","avg_delay")
tram_types_stops = tram_types_stops.withColumnRenamed("avg(delay)","avg_delay")

In [ ]:
#transport_avg_delays.write.parquet('/user/%s/final/parquet/transport_avg_delays' % username)
#hour_avg_delays.write.parquet('/user/%s/final/parquet/hour_avg_delays' % username)

train_types.write.mode('overwrite').parquet('/user/%s/final/parquet/train_types' % username)
bus_types.write.mode('overwrite').parquet('/user/%s/final/parquet/bus_types' % username)
tram_types.write.mode('overwrite').parquet('/user/%s/final/parquet/tram_types' % username)

train_types_hour.write.mode('overwrite').parquet('/user/%s/final/parquet/train_types_hour' % username)
bus_types_hour.write.mode('overwrite').parquet('/user/%s/final/parquet/bus_types_hour' % username)
tram_types_hour.write.mode('overwrite').parquet('/user/%s/final/parquet/tram_types_hour' % username)

train_types_stops.write.parquet('/user/%s/final/parquet/train_types_stops' % username)
bus_types_stops.write.parquet('/user/%s/final/parquet/bus_types_stops' % username)
tram_types_stops.write.parquet('/user/%s/final/parquet/tram_types_stops' % username)

In [ ]:
default_delay = df.select(F.mean(F.col('delay')).alias('mean_delay'),).collect()


In [ ]:
default_delay.show()